In [2]:
import pandas as pd

In [3]:
# загружаем файл и удаляем дубликаты
routes = pd.read_csv('Маршруты.txt', sep = ';').drop_duplicates()

In [4]:
# смотрим данные
routes.head()

,Откуда,Куда,День,Поставщик,Стоимость
0,OSS,UUS,3,47,70635
2,OSS,UUS,3,47,73500
4,OSS,UUS,4,47,47365
5,OSS,UUS,4,47,51950
6,OSS,UUS,6,47,62030


In [5]:
# для удобства объединим первые два столбца
routes['from_to'] = routes['Откуда'] +'-'+ routes['Куда']
routes.head()

,Откуда,Куда,День,Поставщик,Стоимость,from_to
0,OSS,UUS,3,47,70635,OSS-UUS
2,OSS,UUS,3,47,73500,OSS-UUS
4,OSS,UUS,4,47,47365,OSS-UUS
5,OSS,UUS,4,47,51950,OSS-UUS
6,OSS,UUS,6,47,62030,OSS-UUS


In [6]:
df_tt = routes[routes['from_to'] == 'ALA-BUS']
df_tt

,Откуда,Куда,День,Поставщик,Стоимость,from_to
39185,ALA,BUS,4,30,62510,ALA-BUS
39186,ALA,BUS,1,30,61130,ALA-BUS
39187,ALA,BUS,1,30,77220,ALA-BUS
39188,ALA,BUS,5,3,51450,ALA-BUS
39189,ALA,BUS,5,3,65990,ALA-BUS
39190,ALA,BUS,2,3,52710,ALA-BUS
39191,ALA,BUS,2,3,67630,ALA-BUS


In [7]:
# проранжируем поставщиков от самого дешевого до самого дорогого в разрезе откуда-куда и дня
routes['rank'] = routes.groupby(['from_to','День'])['Стоимость'].rank()

In [8]:
routes

,Откуда,Куда,День,Поставщик,Стоимость,from_to,rank
0,OSS,UUS,3,47,70635,OSS-UUS,1.0
2,OSS,UUS,3,47,73500,OSS-UUS,2.0
4,OSS,UUS,4,47,47365,OSS-UUS,1.0
5,OSS,UUS,4,47,51950,OSS-UUS,2.0
6,OSS,UUS,6,47,62030,OSS-UUS,1.0
...,...,...,...,...,...,...,...
44943,GDX,OSS,2,36,29900,GDX-OSS,4.0
44944,GDX,OSS,5,36,25510,GDX-OSS,1.0
44945,GDX,OSS,5,36,26940,GDX-OSS,2.0
44946,GDX,OSS,5,36,27360,GDX-OSS,3.0


In [9]:
# выберем самых дешевых поставиков в разрезе откуда-куда и дня
cheapest_routes = routes[routes['rank'] == 1].iloc[:, [5, 3]].copy()

In [10]:
cheapest_routes

,from_to,Поставщик
0,OSS-UUS,47
4,OSS-UUS,47
6,OSS-UUS,47
8,OSS-UUS,47
10,OSS-UUS,47
...,...,...
44924,GDX-OSS,36
44929,GDX-OSS,36
44934,GDX-OSS,36
44939,GDX-OSS,36


In [11]:
# подсчитаем сколько сколько дней в неделю поставщик осказывался самым выгодным
count_provider = pd.DataFrame({'count' : cheapest_routes.groupby(['from_to','Поставщик']).size()}).reset_index().copy()

In [12]:
# проранжируем поставщиков по количеству раз когда поставщик был самый выгодный
count_provider['rank2'] = count_provider.groupby('from_to')['count'].rank(ascending = False)

In [30]:
count_provider['rank3'] = count_provider.sort_values(['count', 'Поставщик'], ascending = [False, True])\
                                        .groupby(['from_to'])\
                                        .cumcount() + 1

In [31]:
count_provider

,from_to,Поставщик,count,rank2,rank3
0,AAQ-DEL,3,2,1.0,1
1,AAQ-IAR,30,2,2.0,2
2,AAQ-IAR,47,5,1.0,1
3,AAQ-KHV,36,4,1.0,1
4,AAQ-KHV,47,3,2.0,2
...,...,...,...,...,...
2221,YWG-MOW,3,7,1.0,1
2222,YYC-MOW,3,7,1.0,1
2223,ZRH-BER,3,4,1.0,1
2224,ZRH-BER,11,3,2.0,2


In [32]:
# выберем поставщиков, которые были выгодные больше всего дней в неделю
result = count_provider[count_provider['rank3'] == 1].iloc[:, [0,1]].copy()

In [33]:
result

,from_to,Поставщик
0,AAQ-DEL,3
2,AAQ-IAR,47
3,AAQ-KHV,36
5,AAQ-KVK,36
6,AAQ-NUX,36
...,...,...
2220,YTO-SFO,3
2221,YWG-MOW,3
2222,YYC-MOW,3
2223,ZRH-BER,3
